# create some data to test intake-qcodes

standard imports...

In [1]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
from pathlib import Path
import numpy as np

Imports and setup for saving and plotting data

In [2]:
# first some imports to handle the data saving
import qcodes as qc
from qcodes import initialise_database
from qcodes.dataset.experiment_container import load_or_create_experiment

db_path = Path('./data/test_dataset.db').absolute()
db_path.parents[0].mkdir(parents=True, exist_ok=True)
qc.config.current_config.core.db_location = str(db_path)

initialise_database()  # just in case no database file exists

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|███████████████████████████████████| 1/1 [00:00<00:00, 334.55it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|███████████████████████████████████| 1/1 [00:00<00:00, 250.81it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v8 -> v9: 100%|███████████████████████████████████| 1/1 [00:00<00:00, 250.71it/s]


### Setup DummyInstr

In [3]:
# and the imports I need to do the measurement
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_dataset

# making some fake instruments
try:
    dac = DummyInstrument('zed', gates=['ch1', 'ch2']) # to set
    dmm = DummyInstrument('bob', gates=['v1', 'v2'])   # to get
except KeyError:
    pass

In [4]:
from functools import partial

# and we'll make a 2D gaussian to sample from/measure
def gauss_model(x0: float, y0: float, sigma: float, noise: float=0.0005):
    """
    Returns a generator sampling a gaussian. The gaussian is
    normalised such that its maximal value is simply 1
    """
    while True:
        (x, y) = yield
        model = np.exp(-((x0-x)**2+(y0-y)**2)/2/sigma**2)*np.exp(2*sigma**2)
        noise = np.random.randn()*noise
        yield (model + noise)
        
# connect the dmm.v1 to "measure" the gaussian

gauss1 = gauss_model(0.1, 0.2, 0.25)
next(gauss1)

gauss2 = gauss_model(-0.2, 0.1, 0.5)
next(gauss2)

def measure_gauss1(dac):
    val = gauss1.send((dac.ch1.get(), dac.ch2.get()))
    next(gauss1)
    return val

def measure_gauss2(dac):
    val = gauss2.send((dac.ch1.get(), dac.ch2.get()))
    next(gauss2)
    return val*1e-9

dmm.v1.get = partial(measure_gauss1, dac)
dmm.v2.get = partial(measure_gauss2, dac)

### sweeps with a 'grid'

In [7]:
def do1d(exp, param_set, xarray, delay, *param_meas):

    meas = Measurement(exp=exp)
    meas.write_period = 0.5
    
    meas.register_parameter(param_set)
    
    output = []
    for pm in param_meas:
        meas.register_parameter(pm, setpoints=(param_set,))
        output.append([pm, None])
        
    with meas.run() as ds:

        for x in xarray:
            param_set.set(x)
            time.sleep(delay)
            for i, parameter in enumerate(param_meas):
                output[i][1] = parameter.get()
            
            ds.add_result((param_set, x),
                                 *output)
    
        return ds.dataset

def do2d_grid(exp, param_set1, xarray, delay1,
              param_set2, yarray, delay2,
              *param_meas):

    meas = Measurement(exp=exp)
    meas.write_period = 0.5
    
    meas.register_parameter(param_set1)
    meas.register_parameter(param_set2)

    output = []
    for parameter in param_meas:
        meas.register_parameter(parameter, setpoints=(param_set1,param_set2))
        output.append([parameter, None])

    with meas.run() as ds:
        
        for y in yarray:
            param_set2.set(y)
            time.sleep(delay2)
            for x in xarray:
                param_set1.set(x)
                time.sleep(delay1)
                for i, parameter in enumerate(param_meas):
                    output[i][1] = parameter.get()
                ds.add_result((param_set1, x),
                              (param_set2, y),
                              *output)
        time.sleep(0.2)
        
        return ds.dataset  

In [18]:
exp = load_or_create_experiment("experiment_2", sample_name="sample_2")

xarr = np.linspace(-2,1,250)
ds = do1d(exp, dac.ch1, xarr, 0.01, dmm.v1, dmm.v2)

xarr = np.linspace(-1,2,15)
yarr = np.linspace(-1,1,10)
ds = do2d_grid(exp, dac.ch1, xarr, 0.01, dac.ch2, yarr, 0.02, dmm.v1, dmm.v2)

xarr = np.linspace(-2,2.5,105)
ds = do1d(exp, dac.ch1, xarr, 0.01, dmm.v1, dmm.v2)

xarr = np.linspace(-2,1,15)
yarr = np.linspace(-2,4,21)
ds = do2d_grid(exp, dac.ch1, xarr, 0.01, dac.ch2, yarr, 0.02, dmm.v1, dmm.v2)

Starting experimental run with id: 11. 
Starting experimental run with id: 12. 
Starting experimental run with id: 13. 
Starting experimental run with id: 14. 


### sweeps with ridiculous spacing

In [16]:
def get_x():
    return np.random.uniform(low=-1.0, high=2.0)

def get_y():
    return np.random.uniform(low=-1.0, high=1.0)

def do2d_rand(exp, param_set1, param_set2, n, delay,
              *param_meas):

    meas = Measurement(exp=exp)
    meas.write_period = 0.5
    
    meas.register_parameter(param_set1)
    param_set1.post_delay = 0
    meas.register_parameter(param_set2)
    param_set2.post_delay = 0

    output = []
    for parameter in param_meas:
        meas.register_parameter(parameter, setpoints=(param_set1,param_set2))
        output.append([parameter, None])

    with meas.run() as ds:
        
        for i in range(n):
                
                x = get_x()
                param_set1.set(x)
                
                y = get_y()
                param_set2.set(y)
                
                time.sleep(delay)
                for i, parameter in enumerate(param_meas):
                    output[i][1] = parameter.get()
                ds.add_result((param_set1, x),
                                     (param_set2, y),
                                     *output)
        time.sleep(0.2)
        
    return ds.run_id  # convenient to have for plotting

In [17]:
ds = do2d_rand(exp, dac.ch1, dac.ch2, 250, 0.01, dmm.v1, dmm.v2)

Starting experimental run with id: 10. 
